In [1]:
import os, pickle, sys, torch, numpy as np

# table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/all_info_datasets/flexible_models_best_FP"
table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/puuting_h_in_the_middle/flexible_models_best_FP_optinal_MW"
# table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/fix_combining_dataset_load_mfp_bug/flexible_models_best_FP"
# table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/no_postional_encoding/flexible_models_best_FP"
table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/stable_argsort/flexible_models_best_FP"

# '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/all_2d1d_datasets/entropy_radius_exps_on_2d1d_datasets'

print_keys = [
    "test_mean_rank_1/",
    # "test_mean_rank_5/",
    # "test_mean_mean_rank",
    "test_mean_cos",
    "test_mean_f1", 
]

all_exps = os.listdir(table_path)
all_exps = sorted(all_exps)
# load pickles 
from collections import defaultdict
exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
for exp in all_exps:
    # print(exp)
    if 'trial' in exp:
        trial_spelling = "trial"
    elif 'trail' in exp:
        trial_spelling = "trail"
        
    with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
        result = pickle.load(f)[0]
      

        
        for print_key in print_keys:
            for pickle_key in result:
                
                if print_key in pickle_key:
                    exp_name_to_key_to_results["_".join(exp.split(trial_spelling)[:-1])][pickle_key].append(result[pickle_key])
            # exp_name_to_key_to_results[exp][key].append(result[key])
            # print(key, result[key])
    
            
    
exp_name_to_key_to_results


defaultdict(<function __main__.<lambda>()>,
            {'r0_r2_FP_': defaultdict(list,
                         {'test_mean_rank_1/all_inputs': [0.947509765625,
                           0.9439290165901184,
                           0.9437662959098816],
                          'test_mean_rank_1/HSQC_H_NMR': [0.9093424677848816,
                           0.90576171875,
                           0.906494140625],
                          'test_mean_rank_1/HSQC_C_NMR': [0.9414876103401184,
                           0.9456380009651184,
                           0.9353840947151184],
                          'test_mean_rank_1/only_hsqc': [0.8827311396598816,
                           0.877685546875,
                           0.8816731572151184],
                          'test_mean_rank_1/only_1d': [0.9107258915901184,
                           0.9109700322151184,
                           0.9111328125],
                          'test_mean_rank_1/only_H_NMR': [0.63728839159011

In [2]:
avg_results = {}
for exp_name, key_to_results in exp_name_to_key_to_results.items():
    avg_results[exp_name] = {key: np.mean(val) for key, val in key_to_results.items()}
    
avg_results

{'r0_r2_FP_': {'test_mean_rank_1/all_inputs': np.float64(0.945068359375),
  'test_mean_rank_1/HSQC_H_NMR': np.float64(0.9071994423866272),
  'test_mean_rank_1/HSQC_C_NMR': np.float64(0.9408365686734518),
  'test_mean_rank_1/only_hsqc': np.float64(0.8806966145833334),
  'test_mean_rank_1/only_1d': np.float64(0.9109429121017456),
  'test_mean_rank_1/only_H_NMR': np.float64(0.6349826455116272),
  'test_mean_rank_1/only_C_NMR': np.float64(0.8810763756434122),
  'test_mean_cos/all_inputs': np.float64(0.9018896222114563),
  'test_mean_cos/HSQC_H_NMR': np.float64(0.8754668434460958),
  'test_mean_cos/HSQC_C_NMR': np.float64(0.8985201716423035),
  'test_mean_cos/only_hsqc': np.float64(0.8598865667978922),
  'test_mean_cos/only_1d': np.float64(0.875167985757192),
  'test_mean_cos/only_H_NMR': np.float64(0.7734416127204895),
  'test_mean_cos/only_C_NMR': np.float64(0.8557557264963785),
  'test_mean_f1/all_inputs': np.float64(0.9067577521006266),
  'test_mean_f1/HSQC_H_NMR': np.float64(0.88188429

In [3]:
# print in latex format
for table_name in  ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR",  "only_C_NMR"]:
    print("\n\n")
    print ("\\begin{table}[h]")
    print("\centering")
    print('\\begin{tabular}{|c|c|c|c|c|c|}')
    print("\hline")
    print("Model Input & Rank-1$\\uparrow$ & Rank-5$\\uparrow$ & Mean Rank$\\downarrow$ & Cosine Sim$\\uparrow$ & F1-score$\\uparrow$\\\\")
    for exp_name, key_to_results in avg_results.items():
        name = exp_name
        if name[:2]=="FP":
            name = "R"+name.split(" ")[-1][-2]+" FP"
        print(f'{name.replace("_", " ")}', end="")
        for key, val in key_to_results.items():
            if table_name!=key.split("/")[-1]: 
                continue
            # print(key)
            if "test_mean_rank_1" in key or "test/mean_rank_5" in key:
                print(f' & {val*100:.2f}\%', end="")
            elif "test_mean_mean_rank" in key :
                print(f' & {val+1:.2f}', end="")
            else:
                print(f' & {val:.4f}', end="")
        print(" \\\\")
        print("\hline")
        
        
    print('\end{tabular}')
    print("\caption{for " + f'{table_name.replace("_", " ")}'+ ", which entropy-based FP to use}")
    print('\label{table_name}')
    print('\end{table}') 
    




\begin{table}[h]
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
r0 r2 FP  & 94.51\% & 0.9019 & 0.9068 \\
\hline
r0 r3 FP  & 94.19\% & 0.8811 & 0.8885 \\
\hline
r0 r4 FP  & 94.23\% & 0.8704 & 0.8775 \\
\hline
\end{tabular}
\caption{for all inputs, which entropy-based FP to use}
\label{table_name}
\end{table}



\begin{table}[h]
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
r0 r2 FP  & 90.72\% & 0.8755 & 0.8819 \\
\hline
r0 r3 FP  & 90.53\% & 0.8544 & 0.8639 \\
\hline
r0 r4 FP  & 90.35\% & 0.8459 & 0.8553 \\
\hline
\end{tabular}
\caption{for HSQC H NMR, which entropy-based FP to use}
\label{table_name}
\end{table}



\begin{table}[h]
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\